In [11]:
def gapsplit(
        model, n, max_tries=None,
        primary='sequential', primary_tol=0.001,
        secondary_frac=0.05,
        fva=None,
        min_range=1e-5,
        enforce_range=True,
        report_interval=0.1,
        quiet=False):
    """Randomly sample a COBRA model.

    Parameters
    ----------
    model: cobra.Model
        The model to sample. The model will not be modified during sampling.
    n: integer
        Number of samples to generate
    max_tries: integer, optional, default=None
        Sampling attempts that return infeasible or unbounded solutions are
        discarded. Thus the total number of optimizations may exceed `n` for
        difficult models. `max_tries` limits the total number of attempts. If
        None (default), gapsplit will continue until `n` feasible samples are
        found.
    primary: str, optional, default='sequential'
        Strategy for selection the primary target. Targets are chosen
        sequentially ('sequential', default), randomly ('random'), or by always
        targeting the variable with the largest relative gap ('max').
    primary_tol: float, optional, default=0.001
        The primary target is split by setting the upper and lower bounds to
        the midway point of the max gap. The bounds are set to within +/-
        `primary_tol` times the width of the gap to avoid infeasible solutions
        due to numerical issues.
    secondary_frac: float, optional, default=0.05
        Fraction of model variables randomly chosen as secondary targets during
        each iteration. Default is 0.05 (5% of reactions). If 0, no secondary
        targeting is used; this may decrease coverage but improves runtime for
        numerically difficult models.
    fva: pandas.DataFrame, optional, default=None
        gapsplit uses flux variability analysis (FVA) to find the feasible
        ranges for each variable. The user can supply the output of a previous
        `cobra.flux_analysis.flux_variability_analysis` run to avoid re-running
        FVA. If None (default), gapsplit will run FVA.
    min_range: float, optional, default=1e-5
        Variables are targeted only if their feasible range is larger than
        this value.
    enforce_range: boolean, optional, default=True
        If true (default), round solutions to fall within the feasible range.
        This prevents small deviations outside the feasible range from causing
        small decreases in coverage.
    report_interval: float or int, optional, default=0.1
        Show the coverage and gap statistics at this interval. If a number
        between 0.0 and 1.0 is given, gapsplit reports when that fraction of
        `n` samples is finished (i.e. if N=1000 and reportInterval=0.1, reports
        are printed every 100 samples.) To turn off reporting, set to 0.
    quiet: boolean, optional, default=True
        Set to false to keep gapslit from printing status updates.

    Returns
    -------
    pandas.DataFrame
        A data frame with rows = samples and columns = reactions. This is the
        same format as the other cobrapy samplers.
    """
    # output has rows = samples, columns = variables
    # cobrapy returns a pandas DF

    if quiet:
        report = lambda s: None
    else:
        report = lambda s: print(s)

    reactions = model.reactions

    if fva is None:
        report("Calculating feasible ranges using FVA.")
        fva = cobra.flux_analysis.flux_variability_analysis(
                model, reactions, fraction_of_optimum=0.0)
    else:
        report("Using supplied FVA ranges.")

    if secondary_frac >= 1.0:
        n_secondary = secondary_frac
    else:
        n_secondary = np.floor(secondary_frac * len(model.reactions)).astype(int)

    # only split reactions with feasible range >= min_range
    idxs = (fva.maximum - fva.minimum >= min_range).to_numpy().nonzero()[0]
    weights = (1/(fva.maximum - fva.minimum)**2).to_numpy()

    report("Targeting {}/{} unblocked primary variables.".format(len(idxs), len(model.reactions)))
    report("Targeting {} secondary variables.".format(n_secondary))

    report_header, report_format = _make_report_header(n)
    report("\n" + report_header)
    if report_interval < 1.0:
        report_interval = np.floor(report_interval * n).astype(int)

    samples = np.zeros((n, len(model.reactions)))
    k = 0
    infeasible_count = 0
    if primary == 'sequential':
        # primary_var will increment
        primary_var = -1
    try_ = 0
    start_time = time.time()
    while True:
        if max_tries is not None and try_ >= max_tries:
            break
        try_ += 1
        relative, target, width = _maxgap(samples[0:k,idxs], fva.iloc[idxs,:])
        if primary == 'max':
            primary_var = np.argmax(relative)
        elif primary == 'random':
            primary_var = np.random.choice(len(idxs), 1).astype(int)[0]
        elif primary == 'sequential':
            primary_var += 1
            if primary_var >= len(idxs):
                primary_var = 0

        primary_target = target[primary_var]
        primary_lb = primary_target - primary_tol*width[primary_var]
        primary_ub = primary_target + primary_tol*width[primary_var]

        secondary_vars = np.random.choice(len(idxs), n_secondary, replace=False)
        secondary_targets = target[secondary_vars]
        secondary_weights = weights[idxs[secondary_vars]]

        new_sample = _generate_sample(
            model, idxs[primary_var], primary_lb, primary_ub,
            idxs[secondary_vars], secondary_targets, secondary_weights)
        if new_sample is not None:
            if enforce_range:
                new_sample[new_sample > fva.maximum] = fva.maximum[new_sample > fva.maximum]
                new_sample[new_sample < fva.minimum] = fva.minimum[new_sample < fva.minimum]

            samples[k,:] = new_sample
            k += 1
            if k % report_interval == 0:
                elapsed = time.time() - start_time
                remaining = elapsed / k * (n - k)
                report(report_format.format(
                        i=k, n=n, cov=100*(1-np.mean(relative)),
                        min=np.min(relative), med=np.median(relative),
                        max=np.max(relative), ela=elapsed, rem=remaining,
                        inf=infeasible_count))
        else:
            infeasible_count += 1

        if k >= n: break

    if k < n:
        # max_tries reached; return fewer than n samples
        samples = samples[:k,:]

    return pd.DataFrame(data=samples,columns=fva.maximum.index)

In [12]:
def _generate_sample(
        model, primary_var, primary_lb, primary_ub,
        secondary_vars=None, secondary_targets=None, secondary_weights=None):
    """Formulate a [MI]QP to find a single solution."""
    with model:
        model.reactions[primary_var].lower_bound = primary_lb
        model.reactions[primary_var].upper_bound = primary_ub

        if secondary_vars is not None:
            quad_exp = 0
            for i, sec in enumerate(secondary_vars):
                diff = model.problem.Variable('difference_{}'.format(sec))
                cons = model.problem.Constraint(
                    model.reactions[sec].flux_expression - diff,
                    lb=secondary_targets[i], ub=secondary_targets[i])
                model.add_cons_vars([diff, cons])
                quad_exp += secondary_weights[i] * diff**2
            quad_obj = model.problem.Objective(quad_exp, direction='min')
            model.objective = quad_obj
        else:
            model.objective = model.problem.Objective(0)

        try:
            solution = model.optimize()
        except:
            return None
        
        if solution.status != 'optimal':
            return None
        
        return solution.fluxes

In [13]:
def _maxgap(points, fva=None):
    # points has rows = samples, columns = variables

    # make a copy because we're going to sort the columns
    points = points.copy()
    if fva is not None:
        points = np.vstack((fva.minimum, points, fva.maximum))
    points.sort(0)

    gaps = points[1:,:] - points[0:-1,:]
    width = gaps.max(0)
    loc = gaps.argmax(0)
    left = np.zeros(width.size)
    for i in range(width.size):
        left[i] = points[loc[i],i]
    relative = width / (points[-1,:] - points[0,:])
    target = left + width/2

    return relative, target, width

In [14]:
def _make_report_header(maxN):
    """Return the header and format string for reporting coverage."""
    nw = len(str(maxN))
    frac_width = 2*nw + 1  # width of 300/1000
    frac_header = 'Sample'
    frac_format = '{i:' + str(nw) + 'd}/{n:' + str(nw) + 'd}'
    if frac_width < len(frac_header):
        pad = ''.join([' ' for _ in range(len(frac_header) - frac_width)])
        frac_format = pad + frac_format
    elif len(frac_header) < frac_width:
        pad = ''.join([' ' for _ in range(frac_width - len(frac_header))])
        frac_header = pad + frac_header
    hdr = frac_header + "   Coverage   MinGap   Median   MaxGap     Elapsed     Remaining   Infeasible"
    fmt = frac_format + "    {cov:6.2f}%   {min:6.4f}   {med:6.4f}   {max:6.4f}   {ela:9.2f}     {rem:9.2f}   {inf:10d}"
    return hdr, fmt

In [15]:
# convert a model to graph
def convert_from_model_to_graph(model, name, fba, element_to_trace, cofactors):
    
    # reaction and metabolite number
    num_mets = len(model.metabolites); 
    num_rxns = len(model.reactions);
    
    # creats a .net file
    fileID = open(name, 'w');

    # initialize arc number
    num_arc = 0;

    # write node id
    print("number of nodes = %d" % (num_mets))
    fileID.write('*Vertices %d\n' % num_mets)

    global_id = {}
    for i, met in enumerate(model.metabolites):
        fileID.write(str(i+1) + ' "' + met.id + '"\n')
        # assign a global id for each metabolite
        global_id[met.id] = i+1

    # count total number of arcs
    for i, rxn in enumerate(model.reactions):
    
        # exclude biomass reaction
        biomassRxn = [rxn.id for rxn in model.reactions if 'biomass' in rxn.id]

        # cleans reaction with "objective"
        objRxn = [rxn.id for rxn in model.reactions if 'objective' in rxn.id]
        if len(biomassRxn)==0 and len(objRxn)==0:
            biomassRxn = False
        else:
            biomassRxn = True

        if (fba[rxn.id] == 0 or biomassRxn):
            # controls for active reactions
            continue 
        else:
            # get stoichiometric coefficients
            metPos = [met.id for met in rxn.metabolites if rxn.get_coefficient(met.id)>0] # products
            metNeg = [met.id for met in rxn.metabolites if rxn.get_coefficient(met.id)<0] # reactants

            # cleans graph from demand and sink reactions
            if len(metPos)==0 or len(metNeg)==0:
                continue

            for met_j in metNeg:
                formula_j = model.metabolites.get_by_id(met_j).name.split('_')[-1]
                for met_k in metPos:
                    formula_k = model.metabolites.get_by_id(met_k).name.split('_')[-1]
                    if element_to_trace in formula_j and element_to_trace in formula_k and met_j not in cofactors and met_k not in cofactors:
                        num_arc = num_arc + 1


    # write arc number
    print('number of arcs = %d' % num_arc)
    fileID.write('*Arcs %d\n' % num_arc);

    for i, rxn in enumerate(model.reactions):
        biomassRxn = [rxn.id for rxn in model.reactions if 'biomass' in rxn.id]
        objRxn = [rxn.id for rxn in model.reactions if 'objective' in rxn.id]

        if len(biomassRxn)==0 and len(objRxn)==0:
            biomassRxn = False
        else:
            biomassRxn = True

        if (fba[rxn.id] == 0 or biomassRxn):
            # controls for active reactions
            continue 
        else:
            # get stoichiometric coefficients
            metPos = [met.id for met in rxn.metabolites if rxn.get_coefficient(met.id)>0] # products
            metNeg = [met.id for met in rxn.metabolites if rxn.get_coefficient(met.id)<0] # reactants

            if len(metPos)==0 or len(metNeg)==0:
                continue

            for met_j in metNeg:
                formula_j = model.metabolites.get_by_id(met_j).name.split('_')[-1]
                for met_k in metPos:
                    formula_k = model.metabolites.get_by_id(met_k).name.split('_')[-1]
                
                    # take into account flux value and directionality
                    if element_to_trace in formula_j and element_to_trace in formula_k and met_j not in cofactors and met_k not in cofactors:
                        if fba[rxn.id] > 0:
                            # write arcs in forward direction
                            fileID.write(str(global_id[met_j])+' '+str(global_id[met_k])+' '+str(abs(fba[rxn.id]))+' id '+rxn.id+'\n')
                        else:
                            # write arcs in reverse direction
                            fileID.write(str(global_id[met_k])+' '+str(global_id[met_j])+' '+str(abs(fba[rxn.id]))+' id '+rxn.id+' \n')
                        num_arc = num_arc + 1
                    
    fileID.close()

In [16]:
def test_flux_through_given_metabolites(model,fluxsol,met2check,flux_thres):
    pass_all = True
    for index, met_id in enumerate(met2check):
        pass_index = False
        for rxn in model.metabolites.get_by_id(met_id).reactions:
            if abs(fluxsol[rxn.id]) >= flux_thres:
                pass_index = True
                #print('met id = %s, flux = %2.2f'%(met_id,fluxsol[rxn.id]))
                break
        if not pass_index:
            for rxn in model.metabolites.get_by_id(met_id).reactions:
                print(fluxsol[rxn.id])
            pass_all = False
            break
        
    if pass_all:
        print('success')
    else:
        print('fail for', met_id)

In [17]:
def assign_logL_of_graph(filename):
    M = nx.read_pajek(filename)

    # combine fluxes among equivalent edges
    G = nx.DiGraph()
    for u,v,data in M.edges(data=True):
        w = data['weight']
        if G.has_edge(u,v):
            G[u][v]['weight'] += w
        else:
            G.add_edge(u, v, weight=w)
        
    # compute logL of each edge (-log(normalized probability of influx * normalized probability of outflux))
    logL = {}
    for edge in G.edges():
        node1, node2 =  edge
    
        node1_outflux = 0
        #print(node1)
        for e1 in G.out_edges(node1):
            #print(e1)
            node1_outflux += G.get_edge_data(*e1)['weight']
        prob_out = G.get_edge_data(*edge)['weight']/node1_outflux
        
        node2_influx = 0
        #print(node2)
        for e2 in G.in_edges(node2):
            #print(e2)
            node2_influx += G.get_edge_data(*e2)['weight']
        prob_in = G.get_edge_data(*edge)['weight']/node2_influx
    
        #print(prob_out, prob_in)
        logL[edge] = -np.log(prob_out*prob_in)

    nx.set_edge_attributes(G, logL, 'logL')    
    return G

In [18]:
def compute_shortest_distance(network, met2check,verbose=False):
    import networkx as nx
    
    num_mets = len(met2check)
    shortest_distance = np.ones((num_mets,num_mets))*np.inf
    for index1,met_id1 in enumerate(met2check):
        for index2, met_id2 in enumerate(met2check):
            if index2 < index1:
                continue
        
            # met_id1 -> met_id2
            if verbose:
                print('\nFrom %s to %s' % (met_id1, met_id2))
            if nx.has_path(network, source=met_id1, target=met_id2):
                spath_forward = nx.shortest_path(network, source=met_id1, target=met_id2, weight='logL')
                sdist_forward = nx.dijkstra_path_length(network, source=met_id1, target=met_id2, weight='logL')
                if verbose:
                    print('shortest path (network distance = %2.2f): '%(sdist_forward), end='')
                    for index3, node in enumerate(spath_forward):
                        if index3 == len(spath_forward)-1:
                            print(node)
                        else:
                            print(node,'->',end='')
            else:
                sdist_forward = np.inf
                if verbose:
                    print('no path found in metabolic network from %s to %s' % (met_id1, met_id2))
            shortest_distance[index1][index2] = sdist_forward
        
            # met_id2 -> met_id1
            if verbose:
                print('From %s to %s' % (met_id2, met_id1))
            if nx.has_path(network, source=met_id2, target=met_id1):
                spath_reverse = nx.shortest_path(network, source=met_id2, target=met_id1, weight='logL')
                sdist_reverse = nx.dijkstra_path_length(network, source=met_id2, target=met_id1, weight='logL')
                if verbose:
                    print('shortest path (network distance = %2.2f): '%(sdist_reverse), end='')
                    for index3, node in enumerate(spath_reverse):
                        if index3==len(spath_reverse)-1:
                            print(node)
                        else:
                            print(node,'->',end='')
            else:
                sdist_reverse = np.inf
                if verbose:
                    print('no path found in metabolic network from %s to %s' % (met_id2, met_id1))         
            shortest_distance[index2][index1] = sdist_reverse
            
    df_shortest_distance = pd.DataFrame(shortest_distance, index=met2check, columns=met2check)
    return df_shortest_distance

In [19]:
def construct_tidy_comparison_table(met2check, df_dist_data, df_dist_network):
    res = []
    for index1,met_id1 in enumerate(met2check):
        for index2, met_id2 in enumerate(met2check):
            if index2 < index1:
                continue
            else:
                res.append([met_id1,
                            met_id2,
                            df_dist_data.loc[met_id1,met_id2],
                            df_dist_network.loc[met_id1,met_id2],
                            df_dist_network.loc[met_id2,met_id1],
                            min(df_dist_network.loc[met_id1,met_id2],df_dist_network.loc[met_id2,met_id1])])
            
    df_dist_tidy = pd.DataFrame(res,columns=['met_id_1',
                                             'met_id_2',
                                             'data_dist',
                                             'network_dist_forward',
                                             'network_dist_reverse',
                                             'network_dist_min'])
    df_dist_tidy = df_dist_tidy[~np.isinf(df_dist_tidy.network_dist_min)]
    return df_dist_tidy